# Predicting Tensile Strength of Composite Materials

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11698246772327639041
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20312045056
locality {
  bus_id: 1
}
incarnation: 28533055535198546
physical_device_desc: "device: 0, name: DML, pci bus id: <undefined>"
xla_global_id: -1
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 15229974720
locality {
  bus_id: 1
}
incarnation: 17732169278860695957
physical_device_desc: "device: 1, name: DML, pci bus id: <undefined>"
xla_global_id: -1
]


In [ ]:
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
import tensorflow as tf

@tf.function
def test_op():
    return tf.linalg.matmul(tf.random.normal((512, 512)), tf.random.normal((512, 512)))

with tf.device('/GPU:0'):
    result = test_op()
    print("Ran on GPU:0")
    print(result)

with tf.device('/GPU:1'):
    result = test_op()
    print("Ran on GPU:1")
    print(result)


## Introduction

The Composite Materials dataset hosted on Kaggle provides a comprehensive collection of data intended to aid in the modeling and prediction of mechanical properties of composite materials. This dataset is particularly relevant for materials scientists and engineers focused on understanding how different constituent components—such as fiber type, matrix material, volume fraction, and manufacturing parameters—affect key mechanical performance indicators like tensile strength and Young’s modulus. It facilitates machine learning and regression-based studies, enabling researchers to explore the relationship between material composition and resultant properties. The dataset serves as a valuable resource for educational projects, industrial material design, and predictive modeling in composite mechanics.
    
The dataset can be used to train machine learning models that Young’s modulus based on constituent properties and processing conditions—helping accelerate material discovery and design. In industrial applications, engineers can utilize the dataset to optimize composite formulations for lightweight structures in aerospace, automotive, and civil engineering.

## CRISP-DM FRAMEWORK

<center>
    <img src = images/crisp.png width = 50%/>
</center>

To frame the task, throughout this project, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Here is a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).

### 1. Business Understanding

**Objective:** Predict one of important mechanical properties, Young’s modulus of composite materials based on their composition and processing parameters.

**Goals:**
- Develop a predictive model to assist material design.
- Identify which factors most strongly influence mechanical performance.
- Support material selection for lightweight, high-strength applications.

### 2. Data Understanding

#### Input:

- **Matrix-filler ratio:** The proportion of the polymer matrix to the filler material in the composite
- **Density, kg/m3:** The mass per unit volume of the composite material.
- **Modulus of elasticity, GPa:** Young’s modulus of the base material or resin, indicating its ability to resist elastic deformation.
- **Amount of hardener, m.%:** Percentage of hardening agent (often a curing agent in thermosets) relative to the total mass.
- **Content of epoxy groups, %:** The percentage of reactive epoxy functional groups in the resin formulation
- **Flash point, C_2:** The lowest temperature at which the resin emits vapors that can ignite in air.
- **Surface density, g/m2:** Mass of material (often fabric or prepreg) per unit surface area.
- **Tensile modulus of elasticity, GPa:** The modulus specifically measured under tensile (pulling) stress.
- **Resin consumption, g/m2:** The amount of resin required per unit area of reinforcement during composite fabrication.

#### Output:
- **Tensile strength, MPa:** Maximum stress the material can withstand under tension before failure.

#### Import Necessary Libraries

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

pd.set_option('display.max_rows', 10)
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
import category_encoders as ce

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance

#### Import Data and Get General Information

In [ ]:
raw_df = pd.read_excel('./data/X_bp.xlsx');
raw_df.head()

In [ ]:
df = raw_df.rename(columns={
    'Соотношение матрица-наполнитель': 'Matrix-filler ratio',
    'Плотность, кг/м3': 'Density, kg/m³',
    'модуль упругости, ГПа': 'Modulus of elasticity, GPa',
    'Количество отвердителя, м.%': 'Amount of hardener, mass %',
    'Содержание эпоксидных групп,%_2': 'Content of epoxy groups, %',
    'Температура вспышки, С_2': 'Flash point, °C',
    'Поверхностная плотность, г/м2': 'Surface density, g/m²',
    'Модуль упругости при растяжении, ГПа': 'Tensile modulus of elasticity, GPa',
    'Прочность при растяжении, МПа': 'Tensile strength, MPa',
    'Потребление смолы, г/м2': 'Resin consumption, g/m²'
})
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

In [ ]:
df.info()

**Comment**: 
- There are 9 columns, and 1023 entries. There is no missing value in dataset
- Mostly numeric columns (float64 or int64), making the dataset suitable for regression and correlation analysis.

In [ ]:
df.describe()

#### SUMMARY

**Matrix-filler ratio**
- Mean: 2.93, indicating that on average, the matrix is almost three times more prevalent than the filler. Range: 0.39 to 5.59, showing diverse compositions across samples.
- Insight: A higher matrix ratio may lead to lower stiffness but increased toughness.

**Density (kg/m³)**:
- Mean: 1975.73 kg/m³
- Std Dev: 73.73 kg/m³ — moderate variability.
- Range: 1731.76 to 2207.77 kg/m³
- Insight: The dataset focuses on medium- to high-density composite materials, suitable for structural applications.

**Modulus of Elasticity (GPa)**
- Mean: 739.92 GPa — quite high, which may reflect the stiffness of constituent materials.
- Range: 2.44 to 1911.54 GPa — wide variation depending on composite formulation.

**Amount of Hardener (mass %)**
- Mean: 110.57%. Range: 17.74% to 198.95%
- Insight: Some formulations use significantly higher hardener content, which may influence curing rate and crosslink density.

**Content of Epoxy Groups (%)**
- Mean: ~22.24%
- Insight: This controls the cross-linking potential and impacts mechanical and thermal performance.

**Flash Point (°C)**
- Mean: 285.88°C
- Range: 100°C to 364.61°C
- Insight: Higher flash points imply safer processing and higher thermal stability.

**Surface Density (g/m²)**
- Mean: 482.73 g/m². Std Dev: 281.31 — high variation.
- Insight: Indicates varied prepreg or fiber areal weights, which affect final part thickness and performance.

**Tensile Modulus of Elasticity (GPa)**
- Mean: 73.33 GPa. Range: 64.05 to 81.44 GPa
- Insight: Moderate variation suggests consistent tensile stiffness across samples.

**Tensile Strength (MPa)**
- Mean: 2466.92 MPa
- Range: 1036.86 to 3630.35 MPa
- Insight: High strength levels, consistent with advanced structural composites.

**Resin Consumption (g/m²)**
- Mean: 218.42 g/m²
- Range: 33.80 to 382.49 g/m²
- Insight: Wide variability may be related to fiber architecture and manufacturing process.

#### Data Analysis

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap of Numerical Features")
plt.tight_layout()
plt.show()

In [ ]:
# Calculate correlation of all features with Tensile strength
corr_with_target = df.corr(numeric_only=True)['Tensile strength, MPa'].drop('Tensile strength, MPa')

# Sort by absolute correlation value for better insight
corr_sorted = corr_with_target.abs().sort_values(ascending=False).to_frame(name='Correlation with Tensile Strength')
corr_sorted['Signed Correlation'] = corr_with_target[corr_sorted.index]
corr_sorted

**Comment**: The correlation analysis reveals that tensile strength has very weak linear relationships with all other numerical input features in the dataset.

**Positive Correlations:**
- Modulus of Elasticity (GPa): +0.042. Suggests a very slight trend that stiffer materials may have slightly higher tensile strength, though this is statistically negligible.

**Negative Correlations:**
- Amount of Hardener (mass %): –0.075
- Density (kg/m³): –0.070
- Flash Point (°C): –0.032
- These show slight negative trends, implying that as these values increase, tensile strength may decrease marginally. However, these values are far too low to suggest a meaningful or actionable relationship.

In [ ]:
# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Histogram for each numeric feature
df.hist(figsize=(15, 12), bins=20, edgecolor='black')
plt.suptitle("Histograms of Numeric Features", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# 2. Pair plot (subset to avoid overcrowding)
sns.pairplot(df[['Tensile strength, MPa','Amount of hardener, mass %','Density, kg/m³',
                'Modulus of elasticity, GPa', 'Flash point, °C', 'Resin consumption, g/m²']])
plt.suptitle("Pair Plot of Key Features", fontsize=16, y=1.02)
plt.show()

**Histograms** showing the distribution of each numeric feature. Most features show moderately normal or skewed distributions, with Tensile Strength and Resin Consumption having wide spreads.

#### Feature Engineering

- To find meaningful ratios among the factors influencing tensile strength in composites, we focus on combining features that relate to reinforcement content, matrix effectiveness, and mechanical performance. Ratios can help normalize effects or reveal proportional trends.

1. **Matrix Efficiency Ratio:** Tensile strength / Matrix-filler ratio
- Interpretation: Strength achieved per unit matrix content.
- A higher value suggests the matrix is effectively transferring load to the filler.

2. **Specific Tensile Strength:** Tensile strength / Density
- Interpretation: Measures how strong the composite is per unit weight.
- Used widely in aerospace and automotive industries for lightweight design.

3. **Modulus-to-Strength Ratio:** Modulus of elasticity / Tensile strength
- Interpretation: Indicates brittleness.
- A higher value implies stiffer but potentially more brittle material.

4. **Fiber Efficiency Ratio:** Tensile strength / Resin consumption
- Interpretation: Evaluates how efficiently the composite converts resin mass into tensile strength.
- Useful for optimizing resin usage vs strength.

5. **Crosslinking Efficiency:** Tensile strength / Amount of hardener
- Interpretation: Higher values could indicate more efficient curing and stronger fiber-matrix interface.

In [ ]:
df["Matrix Efficiency Ratio"] = df["Tensile strength, MPa"] / df['Matrix-filler ratio']
df["Specific Tensile Strength"] = df["Tensile strength, MPa"] / df["Density, kg/m³"]
df["Modulus-to-Strength"] = df['Tensile modulus of elasticity, GPa'] / df['Tensile strength, MPa']
df["Fiber Efficiency Ratio"] = df["Tensile strength, MPa"] / df['Resin consumption, g/m²']
df["Crosslinking Efficiency"] = df["Tensile strength, MPa"] / df['Amount of hardener, mass %']
df["Stiffness-to-Fiber-Loading"] = df['Tensile modulus of elasticity, GPa'] / df['Matrix-filler ratio']

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap of Numerical Features")
plt.tight_layout()
plt.show()

In [ ]:
# Calculate correlation of all features with Tensile strength
corr_with_target = df.corr(numeric_only=True)['Tensile strength, MPa'].drop('Tensile strength, MPa')

# Sort by absolute correlation value for better insight
corr_sorted = corr_with_target.abs().sort_values(ascending=False).to_frame(name='Correlation with Tensile Strength')
corr_sorted['Signed Correlation'] = corr_with_target[corr_sorted.index]
corr_sorted.head(10)

In [ ]:
# 2. Pair plot (subset to avoid overcrowding)
sns.pairplot(df[['Tensile strength, MPa','Specific Tensile Strength','Modulus-to-Strength', 
                 'Crosslinking Efficiency', 'Fiber Efficiency Ratio','Matrix Efficiency Ratio']])
plt.suptitle("Pair Plot of Key Features", fontsize=16, y=1.02)
plt.show()

### 3. Data Preparation

#### Create training and test set

In [ ]:
X = df.drop('Tensile strength, MPa', axis=1)
y = df['Tensile strength, MPa']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42);
X_train.head()

### 4. Modeling

#### 1 - Base Model with Default Linear Regression

In [ ]:
basemodel = Pipeline([
    ('scale', StandardScaler()),
    ('linreg', LinearRegression())]);
basemodel.fit(X_train, y_train);
basemodel.score(X_test, y_test);

base_y_testPred = basemodel.predict(X_test);

base_mse = mean_squared_error(y_test, base_y_testPred);
base_score = basemodel.score(X_test,y_test);
print(f"Mean Squared Error of the base model: {base_mse}")
print(f"Score of the base model: {base_score}")

#### 2 - Apply Polynomial Feature to find optimal degree

In [ ]:
train_mses = []
test_mses = []
test_score = []
for i in range(1,5):
    pipe = Pipeline([
        ('scale', StandardScaler()),
        ('poly', PolynomialFeatures(degree=i, include_bias=False)),
        ('linreg', LinearRegression())
    ])
    pipe.fit(X_train, y_train);
    train_preds = pipe.predict(X_train);
    test_preds = pipe.predict(X_test);
    train_mses.append(mean_squared_error(y_train, train_preds));
    test_mses.append(mean_squared_error(y_test, test_preds));
    test_score.append(pipe.score(X_test,y_test));

best_model_complexity = test_mses.index(min(test_mses)) + 1;

In [ ]:
print(f'The Complexity that minimized Test Error was: {test_mses.index(min(test_mses)) + 1}')
print(f'The Best MSE of Test Error was: {min(test_mses)}')
print(f'The Best Score of Test Set was: {max(test_score)}')
plt.plot(range(1,5), train_mses, '--o', label = 'training error')
plt.plot(range(1,5), test_mses, '--o', label = 'testing error')
plt.xticks(range(1, 5), range(1, 5))
plt.xlabel('Degree Complexity')
plt.ylabel('Mean Squared Error')
plt.legend();

#### 3 - Ridge Regresion Model with GridSearch to find optimal alpha

In [ ]:
param_dict = {'ridge__alpha': [0.0001, 0.001, 0.1, 1.0, 10.0, 100.0, 1000]}

ridgemodel = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
]);

# Create the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(estimator=ridgemodel, param_grid=param_dict, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best alpha value found by GridSearchCV
best_alpha = grid_search.best_params_['ridge__alpha']
print(f"Best alpha value: {best_alpha}")

# Get the best model (Lasso with the best alpha)
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
ridge_mse = mean_squared_error(y_test, y_pred)
ridge_score = best_model.score(X_test,y_test);
print(f"Mean Squared Error of the best model: {ridge_mse}")
print(f"Score of the best model: {ridge_score}")

#### 4 - Random Forest Model + GridSearchCV

In [ ]:
param_dict = {'rf__n_estimators': np.arange(50,300,50)}

rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
]);

# Create the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_dict, cv=5, scoring='r2')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best alpha value found by GridSearchCV
best_n = grid_search.best_params_['rf__n_estimators']
print(f"Best n_estimator value: {best_n}")

# Get the best model (Lasso with the best alpha)
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
rf_mse = mean_squared_error(y_test, y_pred)
rf_score = best_model.score(X_test,y_test);
print(f"Mean Squared Error of the best model: {rf_mse}")
print(f"Score of the best model: {rf_score}")

#### 5 - Gradient Boosting Model + GridSearchCV

In [ ]:
param_dict = {'gb__n_estimators': np.arange(50,300,50)}

gb = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingRegressor(random_state=42))
]);

# Create the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(estimator=gb, param_grid=param_dict, cv=5, scoring='r2')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best alpha value found by GridSearchCV
best_n = grid_search.best_params_['gb__n_estimators']
print(f"Best n_estimator value: {best_n}")

# Get the best model (Lasso with the best alpha)
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
gb_mse = mean_squared_error(y_test, y_pred)
gb_score = best_model.score(X_test,y_test);
print(f"Mean Squared Error of the best model: {gb_mse}")
print(f"Score of the best model: {gb_score}")

### 5. Evaluation

#### Train the final model

In [ ]:
param_dict = {'ridge__alpha': [0.0001, 0.001, 0.1, 1.0, 10.0, 100.0, 1000]}

finalmodel = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
]);

# Create the GridSearchCV object with 5-fold cross-validation
grid = GridSearchCV(estimator=finalmodel, param_grid=param_dict, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid.fit(X_train, y_train)

# Get the best alpha value found by GridSearchCV
best_alpha = grid_search.best_params_['ridge__alpha']
print(f"Best alpha value: {best_alpha}")

# Get the best model
final_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
finalmodel_mse = mean_squared_error(y_test, y_pred)
finalmodel_score = best_model.score(X_test,y_test);
print(f"Mean Squared Error of the final model: {finalmodel_mse}")
print(f"Score of the final model: {finalmodel_score}")

#### Investigate the important of features

In [ ]:
#Calculate the permutation importance of Test Set
results = permutation_importance(final_model, X_test, y_test);
results = pd.DataFrame(data=results.importances_mean.round(4), index=X.columns, columns=['Importance']).sort_values(by='Importance', ascending=False)
results.head(10)

In [ ]:
#Calculate the permutation importance
results = permutation_importance(final_model, X_train, y_train);
results = pd.DataFrame(data=results.importances_mean.round(4), index=X.columns, columns=['Importance']).sort_values(by='Importance', ascending=False)
results.head(10)

### 6. Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.

### 7. Further Studying
